### Assignment 4, Value Iteration

Your homework must be implemented in this Notebook file. 
You can add as many cells as you want. However, you are not allowed to touch the code below the line "=============".


In [15]:
#The following variables must be used in your program. You can change the location values for testing purposes
stoneLocation=[2, 8]  #[row, column]
positiveTerminalLocation = [0,9,2]  #[row, column, reward value]
negativeTerminalLocation = [1,9, -2]  #[row, column, reward value]
iteration=30
noise=0.15
discount=0.91

In [1]:
def printBoard(board):
    
    board_top_row = [x for x in range(len(board[0]))]
    
    # print column index
    print('  ', end = ' ')
    for i in range(len(board[0])):# top row
        if i != (len(board[0])-1):
            if i < 10:
                print('\t ', board_top_row[i], end = ' ')
            else:
                print('\t ', board_top_row[i], end = '')
        else:
            if i < 10:
                print('\t ', board_top_row[i])
            else:
                print('\t ', board_top_row[i])
        
    # print row index
    for z in range(len(board)):
        if z < 10:
            print('',board_top_row[z], end = '\t')
        else:
            print(board_top_row[z], end = '\t')

        # print position data
        for x in range(len(board[0])): 
            if x != (len(board[0])-1):
                if x < 10:
                    if x != 0:
                        print('\t', board[z][x].getValue(), '(' , board[z][x].getDirection(), ')', end = '', sep = '')    
                    else:
                        print( '', board[z][x].getValue(), '(' , board[z][x].getDirection(), ')', end = '', sep = '')    
                else:
                    print('\t', board[z][x].getValue(), '(' , board[z][x].getDirection(), ')', end = '', sep = '')   
            else:
                if x < 10:
                    print('\t', board[z][x].getValue(), '(' , board[z][x].getDirection(), ')', '', sep = '')
                else:
                    print('\t', board[z][x].getValue(), '(' , board[z][x].getDirection(), ')', sep = '')                        


In [2]:
class GridWorld():
    def __init__(self):
        self.value = 0.00
        self.direction = 'n'
        self.reward = False
        self.directionValue = [] # n, e, s, w
        
    def setValue(self, updateValue):
        self.value = updateValue
    
    def getValue(self):
        return self.value
    
    def setDirection(self, updateDirection):
        self.direction = updateDirection
        
    def getDirection(self):
        return self.direction
    
    def setReward(self):
        self.reward = True
    
    def getReward(self):
        return self.reward
    
    def addDirectionValue(self, value):
        self.directionValue.append(value)
    
    def getDirectionValue(self):
        return self.directionValue

In [3]:
def addTerminalLocation(grid, row, colume, value):
    grid[row][colume].setValue(value)
    grid[row][colume].setDirection('')
    grid[row][colume].setReward()
    
    # update around terminal policy
    # up Location
    if row < len(grid) and row > 0:
        #print('check up Location')
        if grid[row-1][colume].getReward() == False:
            if value > 0:
                grid[row-1][colume].setDirection('s')
            if value < 0:
                grid[row-1][colume].setDirection('n')
    
    # right Location
    if colume < len(grid[0])-1 and colume >= 0:
        #print('check right Location')
        if grid[row][colume+1].getReward() == False:
            if value > 0:
                grid[row][colume+1].setDirection('w')
            if value < 0:
                grid[row][colume+1].setDirection('e')
    
    # down Location
    if row < len(grid)-1 and row >= 0:
        #print('check down Location')
        if grid[row+1][colume].getReward() == False:
            if value > 0:
                grid[row+1][colume].setDirection('n')
            if value < 0:
                grid[row+1][colume].setDirection('s')
    
    # left Location
    if colume < len(grid[0]) and colume > 0:
        #print('check left Location')
        if grid[row][colume-1].getReward() == False:
            if value > 0:
                grid[row][colume-1].setDirection('e')
            if value < 0:
                grid[row][colume-1].setDirection('w')
    

In [4]:
def checkInRagne(grid, row, colume, checkTarget):
    #print('\t\tlen(grid): ', len(grid)) #3
    #print('\t\tlen(grid[0]):', len(grid[0])) #4
    # up Location
    if row < len(grid)-1 and row >= 0 and checkTarget == 0:
        #print('\t\\ttvalid up Location')
        return True
    
    # right Location
    elif colume < len(grid[0]) and colume >= 0 and checkTarget == 1:
        #print('\t\t\tvalid right Location')
        return True

    # down Location
    elif row < len(grid)-1 and row >= 0 and checkTarget == 2:
        #print('\t\t\tvalid down Location')
        return True
    
    # left Location
    elif colume < len(grid[0])-1 and colume >= 0 and checkTarget == 3:
        #print('\t\t\tvalid left Location')
        return True
      
    #print('\t\t\tfail')
    #print('\t\t\trow:', row)
    #print('\t\t\tcolume:', colume)
    #print('\t\t\tcheckTarget:', checkTarget)
    
    return False

In [5]:
def zeroAndStoneNotAround(grid, row, colume):
    # up Location
    if checkInRagne(grid, row-1, colume, 0):
        #print('\t\t\tcheck up Location')
        if grid[row-1][colume].getValue() != 0 and grid[row-1][colume].getValue() != 'STONE':
           return True
    
    # right Location
    if checkInRagne(grid, row, colume+1, 1):
        #print('\t\t\tcheck right Location')
        if grid[row][colume+1].getValue() != 0 and grid[row][colume+1].getValue() != 'STONE':
           return True

    # down Location
    if checkInRagne(grid, row+1, colume, 2):
        #print('\t\t\tcheck down Location')
        if grid[row+1][colume].getValue() != 0 and grid[row+1][colume].getValue() != 'STONE':
           return True
    
    # left Location
    if checkInRagne(grid, row, colume-1, 3):
        #print('\t\t\tcheck left Location')
        if grid[row][colume-1].getValue() != 0 and grid[row][colume-1].getValue() != 'STONE':
           return True
           
    return False

In [6]:
def calculation(row, colume, grid, probability, discount):
    return (probability)*discount*grid[row][colume].getValue()

In [7]:
def calculateValueIteration (row, colume, grid, noise, discount):
    desiredDirection = 0
    right = 0
    left = 0
    
# pointing up
    if grid[row][colume].getDirection() == 'n':
        # up
        if row == 0 or grid[row-1][colume].getValue() == 'STONE': # top row or stone, point back to himself
            desiredDirection = calculation(row, colume, grid, 1-noise, discount)
        else:
            desiredDirection = calculation(row-1, colume, grid, 1-noise, discount)

        # right
        if colume == len(grid[0])-1 or grid[row][colume+1].getValue() == 'STONE': # right most colume, point back to himself
            right = calculation(row, colume, grid, noise/2, discount)
        else:
            right = calculation(row, colume+1, grid, noise/2, discount)

        # left
        if colume == 0 or grid[row][colume-1].getValue() == 'STONE': # left most colume, point back to himself
            left = calculation(row, colume, grid, noise/2, discount)
        else:
            left = calculation(row, colume-1, grid, noise/2, discount)

    grid[row][colume].addDirectionValue(float(round(desiredDirection + right + left, 2)))
                                       
# pointing right
    # right
    if colume == len(grid[0])-1 or grid[row][colume+1].getValue() == 'STONE': # right most colume, point back to himself
        desiredDirection = calculation(row, colume, grid, 1-noise, discount)
    else:
        desiredDirection = calculation(row, colume+1, grid, 1-noise, discount)
       
    # down
    if row == len(grid)-1 or grid[row+1][colume].getValue() == 'STONE': # buttom row or stone, point back to himself
        right = calculation(row, colume, grid, noise/2, discount)
    else:
        right = calculation(row+1, colume, grid, noise/2, discount)
        
    # up
    if row == 0 or grid[row-1][colume].getValue() == 'STONE': # top row or stone, point back to himself
        left = calculation(row, colume, grid, noise/2, discount)
    else:
        left = calculation(row-1, colume, grid, noise/2, discount)
      
    grid[row][colume].addDirectionValue(float(round(desiredDirection + right + left, 2)))

# pointing down
    # down
    if row == len(grid)-1 or grid[row+1][colume].getValue() == 'STONE': # buttom row or stone, point back to himself
        desiredDirection = calculation(row, colume, grid, 1-noise, discount)
    else:
        desiredDirection = calculation(row+1, colume, grid, 1-noise, discount)

    # left
    if colume == 0 or grid[row][colume-1].getValue() == 'STONE': # left most colume, point back to himself
        right = calculation(row, colume, grid, noise/2, discount)
    else:
        right = calculation(row, colume-1, grid, noise/2, discount)

    # right
    if colume == len(grid[0])-1 or grid[row][colume+1].getValue() == 'STONE': # right most colume, point back to himself
        left = calculation(row, colume, grid, noise/2, discount)
    else:
        left = calculation(row, colume+1, grid, noise/2, discount)

    grid[row][colume].addDirectionValue(float(round(desiredDirection + right + left, 2)))
    

    # pointing left
    # left
    if colume == 0 or grid[row][colume-1].getValue() == 'STONE': # left most colume, point back to himself
        desiredDirection = calculation(row, colume, grid, 1-noise, discount)
    else:
        desiredDirection = calculation(row, colume-1, grid, 1-noise, discount)

    # up
    if row == 0 or grid[row-1][colume].getValue() == 'STONE': # top row or stone, point back to himself
        right = calculation(row, colume, grid, noise/2, discount)
    else:
         right = calculation(row-1, colume, grid, noise/2, discount)
    
    # down
    if row == len(grid)-1 or grid[row+1][colume].getValue() == 'STONE': # buttom row or stone, point back to himself
        left = calculation(row, colume, grid, noise/2, discount)
    else:
        left = calculation(row+1, colume, grid, noise/2, discount)

    grid[row][colume].addDirectionValue(float(round(desiredDirection + right + left, 2)))
    

In [8]:
def getLargerPointingValue(grid, row, colume, valueIndex):
    if valueIndex == 0: #n
        if checkInRagne(grid, row-1, colume, valueIndex):
            #print('get up value')
            return grid[row-1][colume].getValue()

    elif valueIndex == 1: #e
        if checkInRagne(grid, row, colume+1, valueIndex):
            #print('get right value')
            return grid[row][colume+1].getValue()

    elif valueIndex == 2: #s
        if checkInRagne(grid, row+1, colume, valueIndex):
            #print('get down value')
            return grid[row+1][colume].getValue()

    elif valueIndex == 3: #w
        if checkInRagne(grid, row, colume-1, valueIndex):
            #print('get left value')
            return grid[row][colume-1].getValue()
    
    return grid[row][colume].getValue()

In [9]:
import time          # timer

def start():
    #stoneLocation=[2, 8]  #[row, column]
    #positiveTerminalLocation = [0,9,2]  #[row, column, reward value]
    #negativeTerminalLocation = [1,9, -2]  #[row, column, reward value]
    
    # There are four actions west, east, north, south. The actions are all “north” in the initial
    # policy. After each iteration, the actions in the policy need to be updated to the direction
    # that points to the adjacent grid that has the highest value.
    
    classExample = False
    actionList = [ 'n', 'e', 's', 'w']
    debug = False
    numIterations = 0
    iteration = 100
    
    if classExample == False:
        noise=0.15 #(if you go north, 0.85 goes North, 0.075 goes to East or West, 0 goes south.),
        discount=0.91
        size = 10;

        # load the basic information
        grid =[[GridWorld() for x in range(size)] for y in range(size)] 
        
        startTimer = time.time()
    # 0 Iterations
        print('\t\tValues after', numIterations, 'Iterations')
        numIterations = 0
        printBoard(grid)
        print()
        
        numIterations += 1

    # 1 Iterations
        print('\t\tValues after', numIterations, 'Iterations')
        # set stone, positiveTerminalLocation, negativeTerminalLocation
        grid[2][8].setValue('STONE')
        grid[2][8].setDirection('')

        addTerminalLocation(grid, 0, 9, 2.0)
        addTerminalLocation(grid, 1, 9, -2.0)

        # display
        
        printBoard(grid)
    
    else:
        noise=0.2
        discount=0.9
        numRow = 4
        numColume = 3

        grid =[[GridWorld() for x in range(numRow)] for y in range(numColume)] 
        
        startTimer = time.time()
    # 0 Iterations
        print('\t\tValues after', numIterations, 'Iterations')
        printBoard(grid)
        print()
        numIterations += 1

    # 1 Iterations
        print('\t\tValues after', numIterations, 'Iterations')
        grid[1][1].setValue('STONE')
        grid[1][1].setDirection('')

        addTerminalLocation(grid, 0, 3, 1.0)
        addTerminalLocation(grid, 1, 3, -1.0)

        numIterations += 1
        printBoard(grid)

# 1 - max Iterations
    for i in range(iteration-2):
        print('\t\tValues after', i+2, 'Iterations')
        
        if classExample:
            tampGrid =[[GridWorld() for x in range(numRow)] for y in range(numColume)] 
        else:
            tampGrid =[[GridWorld() for x in range(size)] for y in range(size)] 
        
        for row in range(len(grid)):  
            for colume in range(len(grid[0])):
                tampGrid[row][colume].setValue(grid[row][colume].getValue())
                tampGrid[row][colume].setDirection(grid[row][colume].getDirection())
                if grid[row][colume].getReward():
                    tampGrid[row][colume].setReward()
        
        for row in range(len(grid)):
            if debug:
                print('\trow: ', row)
            for colume in range(len(grid[0])): 
                if tampGrid[row][colume].getValue() != 'STONE' and zeroAndStoneNotAround(tampGrid, row, colume) and tampGrid[row][colume].getReward() == False:
                    calculateValueIteration(row, colume, tampGrid, noise, discount)

                    if debug:
                        print('\t\tcolume: ', colume)
                        print('\t\t\t', tampGrid[row][colume].getDirectionValue())
                    
                # check duplicate bigger value
                    tampDirectionList = tampGrid[row][colume].getDirectionValue()
                    firstBiggerValue = max(tampDirectionList)
                    firstBiggerValueIndex = tampGrid[row][colume].getDirectionValue().index(firstBiggerValue)

                    duplicateIndexList = []
                    seachIndex = firstBiggerValueIndex+1

                    #print('firstBiggerValueIndex:', firstBiggerValueIndex)
                    while seachIndex < len(tampDirectionList):
                        #if debug:
                            #print('\t\t\tseachIndex:', seachIndex)
                            #print('\t\t\ttampDirectionList[firstBiggerValueIndex]:', tampDirectionList[firstBiggerValueIndex])
                            #print('\t\t\ttampDirectionList[seachIndex]:', tampDirectionList[seachIndex])

                        if tampDirectionList[firstBiggerValueIndex] == tampDirectionList[seachIndex]:
                            duplicateIndexList.append(seachIndex)
                            #print()

                        seachIndex += 1

                    if debug:
                        print('\t\t\t', duplicateIndexList)

                    if len(duplicateIndexList) == 0:
                        #print(max(tampGrid[row][colume].getDirectionValue()))
                        #print(actionList[tampGrid[row][colume].getDirectionValue().index(max(tampGrid[row][colume].getDirectionValue()))])
                        grid[row][colume].setValue(max(tampGrid[row][colume].getDirectionValue()))
                        grid[row][colume].setDirection(actionList[tampGrid[row][colume].getDirectionValue().index(max(tampGrid[row][colume].getDirectionValue()))])
                    else:
                    # get the duplicate bigger value with bigger value pointing to
                        firstMaximumDirectionValue = otherMaximumDirectionValue = None
                        
                        for i in range(len(duplicateIndexList)):    
                            firstMaximumDirectionValue = getLargerPointingValue(grid, row, colume, firstBiggerValueIndex)
                            otherMaximumDirectionValue = getLargerPointingValue(grid, row, colume, duplicateIndexList[i])

                            #if debug:
                                #print('\t\t\tfirstMaximumDirectionValue:', firstMaximumDirectionValue)
                                #print('\t\t\totherMaximumDirectionValue:', otherMaximumDirectionValue)
                        # udpate grid world
                            if firstMaximumDirectionValue == 'STONE':
                                firstMaximumDirectionValue = grid[row][colume].getValue()

                            if otherMaximumDirectionValue == 'STONE':
                                otherMaximumDirectionValue = grid[row][colume].getValue()

                            #if debug:
                                #print('\t\t\tfirstMaximumDirectionValue:', firstMaximumDirectionValue)
                                #print('\t\t\totherMaximumDirectionValue:', otherMaximumDirectionValue)

                            if firstMaximumDirectionValue > otherMaximumDirectionValue:
                                #print('load firstMaximumDirectionValue')
                                #print('\t\t\tfirstMaximumDirectionValue:', firstMaximumDirectionValue)
                                #print('\t\t\ttampDirectionList[i]:', tampDirectionList[i])
                                #print('\t\t\tfirstBiggerValueIndex:', firstBiggerValueIndex)
                                #print('\t\t\tactionList[firstBiggerValueIndex]:', actionList[firstBiggerValueIndex])

                                #grid[row][colume].setValue(firstMaximumDirectionValue)                
                                grid[row][colume].setDirection(actionList[firstBiggerValueIndex])
                            else:
                                #print('load otherMaximumDirectionValue')
                                #print('\t\t\tfirstMaximumDirectionValue:', firstMaximumDirectionValue)
                                #print('\t\t\ttampDirectionList[i]:', tampDirectionList[i])
                                #print('\t\t\ti:', i)
                                #print('\t\t\tactionList[duplicateIndexList[i]]:', actionList[duplicateIndexList[i]])

                                #grid[row][colume].setValue(tampDirectionList[i])                
                                grid[row][colume].setDirection(actionList[duplicateIndexList[i]])
                                    
        
        numIterations += 1
        #print('grid')
        printBoard(grid)
        
    endTimer = time.time()
    print( 'Time: ', "{:.2f}".format(endTimer-startTimer), 's')

You can insert as many cells as you want above
You are not Allowed to modify the code below this line.
# ===============================

In [10]:
#you need to implement print_result function to print out the result according to the required format
start()

		Values after 0 Iterations
   	  0 	  1 	  2 	  3 	  4 	  5 	  6 	  7 	  8 	  9
 0	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)
 1	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)
 2	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)
 3	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)
 4	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)
 5	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)
 6	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)
 7	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)
 8	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)
 9	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)

		Values after 1 Iterations
   	  0 	  1 	  2 	  3 	  4 	  5 	  6 	  7 	  8 	  9
 0	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(e)	2.0()
 1	0.0(n)	0.0(n)	0.0(n)	0.0(n)	0.0(

# sample output of one status
## 0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n)
## 0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n)
## 0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),2.00(n),0.00(n),0.00(n),0.00(n)
## 0.00(n),0.00(n),(STONE),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n)
## 0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),-2.00(n),0.00(n),0.00(n),0.00(n)
## 0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n)
## 0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n)
## 0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n)
## 0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n)
## 0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n),0.00(n)

